In [1]:
#!/usr/bin/env python3

import requests
import pandas as pd
import pymysql
import configparser
from datetime import datetime, timedelta

# --- Load Toolforge DB credentials ---
cfg = configparser.ConfigParser()
cfg.read('/data/project/community-activity-alerts-system/replica.my.cnf')
user = cfg['client']['user']
password = cfg['client']['password']

import requests

# Fetch the JSON data
url = "https://meta.wikimedia.org/w/api.php?action=sitematrix&format=json"
response = requests.get(url)
data = response.json()

# Extract and clean URLs
projects = []

sitematrix = data.get("sitematrix", {})
for key, val in sitematrix.items():
    if key in ("count", "specials"):
        continue
    if isinstance(val, dict):
        sites = val.get("site", [])
        for site in sites:
            site_url = site.get("url")
            if site_url:
                cleaned_url = site_url.replace("https://", "")
                projects.append(cleaned_url)

base_url = "https://wikimedia.org/api/rest_v1/metrics/edits/aggregate"
editor_type = "all-editor-types"
page_type = "content"
granularity = "monthly"

# --- Calculate last month's date range ---
today = datetime.utcnow().date().replace(day=1)
last_month_end = today - timedelta(days=1)
last_month_start = last_month_end.replace(day=1)

start = last_month_start.strftime("%Y%m%d")
end = last_month_end.strftime("%Y%m%d")

# --- Connect to Toolforge DB ---
DB_NAME = 's56391__community_alerts'
DB_TABLE = 'edit_counts'

conn = pymysql.connect(
    host='tools.db.svc.wikimedia.cloud',
    user=user,
    password=password,
    database=DB_NAME,
    charset='utf8mb4',
    autocommit=True
)

cursor = conn.cursor()

# --- Ensure table exists ---
create_table_sql = f'''
CREATE TABLE IF NOT EXISTS {DB_TABLE} (
    timestamp DATETIME,
    edit_count INT,
    project VARCHAR(255),
    PRIMARY KEY (timestamp, project)
)
'''
cursor.execute(create_table_sql)

# --- Loop through each project and process ---
for project in projects:
    print(f"Fetching edits for {project} from {start} to {end}")

    url = f"{base_url}/{project}/{editor_type}/{page_type}/{granularity}/{start}/{end}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"API Error for {project}: {response.status_code} - {response.text}")
        continue

    try:
        data = response.json()
        edit_counts = data["items"][0]["results"]
        if not edit_counts:
            print(f"No data returned for {project}")
            continue
    except Exception as e:
        print(f"Parsing error for {project}: {e}")
        continue

    df = pd.DataFrame(edit_counts)
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
    df['project'] = project
    df.rename(columns={'edits': 'edit_count'}, inplace=True)

    for _, row in df.iterrows():
        insert_sql = f"""
        INSERT INTO {DB_TABLE} (timestamp, edit_count, project)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE edit_count = VALUES(edit_count)
        """
        cursor.execute(
            insert_sql,
            (row['timestamp'].to_pydatetime(), int(row['edit_count']), row['project'])
        )

print("All data saved successfully.")

# --- Cleanup ---
cursor.close()
conn.close()


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


ModuleNotFoundError: No module named 'pymysql'